In [ ]:
import pandas as pd
import numpy as np
import json
import os
import random
from datetime import datetime, timedelta
from tqdm.notebook import tqdm

WALLET_ADDRESSES_CSV = 'wallet_addresses.csv'
RAW_TRANSACTIONS_JSON = 'raw_compound_transactions.json'
OUTPUT_SCORES_CSV = 'compound_risk_scores.csv'

if not os.path.exists(WALLET_ADDRESSES_CSV):
    raise FileNotFoundError(f"'{WALLET_ADDRESSES_CSV}' not found. Please upload the CSV containing wallet addresses to Colab.")


def generate_synthetic_transactions(wallet_ids, num_tx_per_wallet_range=(5, 20)):
    """
    Generates synthetic Compound-like transaction data for a list of wallet IDs.
    Includes diverse transaction types and simulates some risky behavior.
    """
    print("Generating synthetic transaction data...")
    all_synthetic_transactions = []

    transaction_types = ['supply', 'borrow', 'repay', 'withdraw']
    assets = ['USDC', 'DAI', 'WETH', 'WBTC']

    start_date = datetime(2023, 1, 1)
    end_date = datetime.now()

    tx_id_counter = 0

    for wallet_id in tqdm(wallet_ids, desc="Generating data for wallets"):
        num_transactions = random.randint(*num_tx_per_wallet_range)
        wallet_transactions = []

        current_timestamp = int(start_date.timestamp())

        has_liquidation = random.random() < 0.15
        for i in range(num_transactions):
            tx_type = random.choice(transaction_types)
            asset_symbol = random.choice(assets)


            amount_usd = round(random.uniform(10, 5000), 2)

            current_timestamp += random.randint(3600 * 24, 3600 * 24 * 30)
            if current_timestamp > int(end_date.timestamp()):
                current_timestamp = int(end_date.timestamp())

            tx_id_counter += 1
            transaction_hash = f"0x{os.urandom(32).hex()}"

            wallet_transactions.append({
                "id": f"{wallet_id}_{tx_id_counter}",
                "txHash": transaction_hash,
                "wallet_id": wallet_id,
                "type": tx_type,
                "amount": str(int(amount_usd * 1e18)),
                "amountUSD": str(amount_usd),
                "asset_symbol": asset_symbol,
                "timestamp": current_timestamp
            })

        if has_liquidation and any(tx['type'] == 'borrow' for tx in wallet_transactions):

            borrow_timestamps = [tx['timestamp'] for tx in wallet_transactions if tx['type'] == 'borrow']
            if borrow_timestamps:
                liquidation_timestamp = random.choice(borrow_timestamps) + random.randint(3600 * 24, 3600 * 24 * 7)
                liquidation_amount_usd = round(random.uniform(5, 50), 2)
                tx_id_counter += 1
                wallet_transactions.append({
                    "id": f"{wallet_id}_{tx_id_counter}_liq",
                    "txHash": f"0x{os.urandom(32).hex()}",
                    "wallet_id": wallet_id,
                    "type": "liquidate_as_borrower",
                    "amount": str(int(liquidation_amount_usd * 1e18)),
                    "amountUSD": str(liquidation_amount_usd),
                    "asset_symbol": random.choice(assets),
                    "timestamp": liquidation_timestamp
                })

        all_synthetic_transactions.extend(wallet_transactions)

    all_synthetic_transactions.sort(key=lambda x: x['timestamp'])

    with open(RAW_TRANSACTIONS_JSON, 'w') as f:
        json.dump(all_synthetic_transactions, f, indent=4)

    print(f"Generated {len(all_synthetic_transactions)} synthetic transactions for {len(wallet_ids)} wallets.")
    print(f"Synthetic data saved to '{RAW_TRANSACTIONS_JSON}'.")
    return all_synthetic_transactions

def main():
    """
    Main function to run the entire risk scoring pipeline using generated synthetic data.
    """
    print("Starting Compound Wallet Risk Scoring pipeline with synthetic data generation...")

    try:
        wallets_df = pd.read_csv(WALLET_ADDRESSES_CSV)
        wallets_df['wallet_id'] = wallets_df['wallet_id'].astype(str).str.lower()
        wallet_ids = wallets_df['wallet_id'].dropna().unique().tolist()
        print(f"Found {len(wallet_ids)} unique wallet addresses from CSV.")
    except Exception as e:
        print(f"Error reading wallet addresses CSV: {e}")
        return

    raw_transactions_list = generate_synthetic_transactions(wallet_ids)

    if not raw_transactions_list:
        print("No synthetic transactions generated. Exiting.")
        return
    df = pd.DataFrame(raw_transactions_list)

    df['amountUSD'] = pd.to_numeric(df['amountUSD'], errors='coerce')
    df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')
    df.dropna(subset=['amountUSD', 'timestamp'], inplace=True)

    print("\nSynthetic transactions DataFrame head:")
    print(df.head())
    print("\nSynthetic transactions DataFrame info:")
    df.info()

    print("\nEngineering Compound-specific features...")

    df['is_liquidated_borrower'] = (df['type'] == 'liquidate_as_borrower')

    wallet_features = df.groupby('wallet_id').agg(
        total_tx_count=('txHash', 'count'),
        first_tx_timestamp=('timestamp', 'min'),
        last_tx_timestamp=('timestamp', 'max'),
        unique_assets_count=('asset_symbol', 'nunique'),

        num_supplies=('type', lambda x: (x == 'supply').sum()),
        total_supply_usd=('amountUSD', lambda x: x[df['type'] == 'supply'].sum()),

        num_borrows=('type', lambda x: (x == 'borrow').sum()),
        total_borrow_usd=('amountUSD', lambda x: x[df['type'] == 'borrow'].sum()),

        num_repays=('type', lambda x: (x == 'repay').sum()),
        total_repay_usd=('amountUSD', lambda x: x[df['type'] == 'repay'].sum()),

        num_withdraws=('type', lambda x: (x == 'withdraw').sum()),
        total_withdraw_usd=('amountUSD', lambda x: x[df['type'] == 'withdraw'].sum()),

        num_liquidations_as_borrower=('is_liquidated_borrower', 'sum'),
        num_liquidations_as_liquidator=('type', lambda x: (x == 'liquidate_as_liquidator').sum())
    )

    wallet_features['wallet_age_days'] = (pd.to_datetime(wallet_features['last_tx_timestamp'], unit='s') - \
                                          pd.to_datetime(wallet_features['first_tx_timestamp'], unit='s')).dt.total_seconds() / (60 * 60 * 24)

    wallet_features['avg_tx_per_day'] = wallet_features['total_tx_count'] / (wallet_features['wallet_age_days'].replace(0, np.nan) + 1e-6)
    wallet_features['avg_tx_per_day'].fillna(wallet_features['total_tx_count'], inplace=True)

    wallet_features['repayment_ratio_usd'] = wallet_features.apply(
        lambda row: row['total_repay_usd'] / row['total_borrow_usd'] if row['total_borrow_usd'] > 0 else 1.0,
        axis=1
    )

    wallet_features['borrow_to_supply_ratio_usd'] = wallet_features.apply(
        lambda row: row['total_borrow_usd'] / row['total_supply_usd'] if row['total_supply_usd'] > 0 else 0.0,
        axis=1
    )
    wallet_features['borrow_to_supply_ratio_usd'] = wallet_features['borrow_to_supply_ratio_usd'].replace([np.inf, -np.inf], np.nan).fillna(0).clip(upper=5.0)

    print("Feature engineering complete.")
    wallet_features_df = wallet_features.reset_index()

    print("\nCalculating risk scores...")

    scores = []
    for _, row in wallet_features_df.iterrows():
        score = 500

        score += np.log1p(row['wallet_age_days']) * 5
        score += np.log1p(row['total_supply_usd']) * 0.5
        score += min(row['repayment_ratio_usd'], 1.0) * 200
        score += min(row['num_repays'], 10) * 5

        score -= row['num_liquidations_as_borrower'] * 300
        score -= min(row['borrow_to_supply_ratio_usd'], 2.0) * 100

        if row['num_supplies'] > 0:
            withdrawal_ratio = row['num_withdraws'] / row['num_supplies']
            score -= min(withdrawal_ratio, 2.0) * 20

        final_score = max(0, min(1000, score)) 
        scores.append(final_score)

    final_scores_df = wallet_features_df[['wallet_id']].copy()
    final_scores_df['score'] = scores
    print("Risk score calculation complete.")

    print(f"\nSaving final scores to {OUTPUT_SCORES_CSV}...")
    final_scores_df.to_csv(OUTPUT_SCORES_CSV, index=False)
    print("Scoring process finished. Output saved.")
    print(f"Please download '{OUTPUT_SCORES_CSV}' from the Colab files section.")
    print("\nFinal Scores DataFrame head:")
    print(final_scores_df.head())

if __name__ == "__main__":
    main()

Starting Compound Wallet Risk Scoring pipeline with synthetic data generation...
Found 103 unique wallet addresses from CSV.
Generating synthetic transaction data...


Generating data for wallets:   0%|          | 0/103 [00:00<?, ?it/s]

Generated 1357 synthetic transactions for 103 wallets.
Synthetic data saved to 'raw_compound_transactions.json'.

Synthetic transactions DataFrame head:
                                                id  \
0    0x104ae61d8d487ad689969a17807ddc338b445416_86   
1   0x9a363adc5d382c04d36b09158286328f75672098_960   
2   0x4e6e724f4163b24ffc7ffe662b5f6815b18b4210_479   
3  0xd0df53e296c1e3115fccc3d7cdf4ba495e593b56_1148   
4   0x1e43dacdcf863676a6bec8f7d6896d6252fac669_214   

                                              txHash  \
0  0x36feabd539bacb8b3eb3dea225749d3d0f5883ddfc80...   
1  0xfc20b9a07ee05f2b2273ed482187f7c1ba1f5876a285...   
2  0xeb7da1456ab8c76141a0dcc43a4596bfd3286310fb9a...   
3  0x73f0404fab4700d4abc27aa6fae80db3b2e0e9260f8c...   
4  0xe65dd060b0452e3dc8d9dd0efdf2ce92a2fce206a028...   

                                    wallet_id      type  \
0  0x104ae61d8d487ad689969a17807ddc338b445416    borrow   
1  0x9a363adc5d382c04d36b09158286328f75672098    supply   
2  0x4e6

/tmp/ipython-input-7-2887926655.py:172: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  wallet_features['avg_tx_per_day'].fillna(wallet_features['total_tx_count'], inplace=True)
